### Component Of RAG Application

#### Create a simple Rag Application

In [ ]:
!pip install -U langchain-chroma
!pip install -U DeepEval

In [43]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain_ollama import ChatOllama
import deepeval
from deepeval.tracing import observe, update_current_span
import os
os.environ["CONFIDENT_TRACE_FLUSH"] = "YES"

@observe(type="llm", model="llama3.2:latest")
def get_local_llm(model_name):
    return ChatOllama(
        base_url="http://localhost:11434",
        model=model_name,
        temperature=0.5,
        max_tokens=500
)

In [61]:
# Load data from Web
from langchain_community.document_loaders import SeleniumURLLoader
loader = SeleniumURLLoader(urls=["https://www.descope.com/learn/post/mcp"])
data = loader.load()
# Convert Data Into Chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               chunk_overlap=150,
                                               separators=["\n\n", "\n", ". ", " ", ""])
docs = text_splitter.split_documents(data)
# Create Embedding and Vector Store
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
# Note: Ensure Ollama is running and has the model available
ollama_embedding = OllamaEmbeddings(model="llama3.2:latest")
vector_store = Chroma.from_documents(documents=docs, 
                                     embedding= ollama_embedding)
## Setting up retrieval augmented generation (RAG)
retriever = vector_store.as_retriever()
## Initialize LLM For Generation
llm = get_local_llm("llama3.2:latest")
# Define Prompt Template 
from langchain_core.prompts import ChatPromptTemplate
template = """
Your are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know, just say you don't know.
keep the answer concise.

Question: {question}
Context: {context}
"""
prompt = ChatPromptTemplate.from_template(template)
# Construct Rag Chain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser, SimpleJsonOutputParser
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser()
)

In [63]:
rag_chain.invoke("How MCP works?")

'The Model Context Protocol (MCP) is a standardized way for LLMs and AI agents to connect with external data sources and tools. It allows the server to return requested information to the client in a standardized format, which is then integrated into the agent\'s understanding of the conversation. This process enables the agent to generate responses based on current data, creating an unobtrusive experience where it appears to "know" information it couldn\'t possibly have from its training data alone.'